# Laboratorio 4 

## Importación de datos

In [1]:
#importación de dataser IMD de Keras
import numpy as np
from keras.datasets import imdb
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout, Input, Concatenate, BatchNormalization
from keras.models import Model
import tensorflow as tf



In [2]:
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=50000)

## Pre-procesamiento

In [3]:
# Secuencie y rellene las críticas para que todas tengan una longitud uniforme.
x_train = sequence.pad_sequences(x_train, maxlen=500)
x_test = sequence.pad_sequences(x_test, maxlen=500)

In [4]:
# Preview de los datos
print(x_train.shape)
print(x_test.shape)



(25000, 500)
(25000, 500)


In [5]:
# Función para extraer características adicionales
def extract_features(sequences):
    lengths = np.array([len(seq) for seq in sequences])  # Longitud de cada crítica
    avg_word_length = np.array([np.mean([len(str(word)) for word in seq]) for seq in sequences])  # Promedio de longitud de palabras
    unique_word_ratio = np.array([len(set(seq)) / len(seq) for seq in sequences])  # Proporción de palabras únicas
    
    return np.vstack((lengths, unique_word_ratio, avg_word_length)).T

In [6]:
# Extraer características de entrenamiento y prueba
x_train_features = extract_features(x_train)
x_test_features = extract_features(x_test)

In [7]:
x_train = np.concatenate((x_train, x_train_features), axis=1)
x_test = np.concatenate((x_test, x_test_features), axis=1)

# Preview de los datos
print(x_train.shape)
print(x_test.shape)



(25000, 503)
(25000, 503)


## Modelo

In [8]:
# Crear modelo
model = Sequential()
model.add(Embedding(50000, 128, input_length=10))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

c:\Users\DIEAL\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [9]:
# Entrenar modelo
model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test))



Epoch 1/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 765s 974ms/step - accuracy: 0.7123 - loss: 0.5536 - val_accuracy: 0.7999 - val_loss: 0.4919
Epoch 2/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 650s 831ms/step - accuracy: 0.8616 - loss: 0.3424 - val_accuracy: 0.8444 - val_loss: 0.3664
Epoch 3/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 551s 705ms/step - accuracy: 0.8999 - loss: 0.2648 - val_accuracy: 0.8168 - val_loss: 0.4161
Epoch 4/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 609s 779ms/step - accuracy: 0.9278 - loss: 0.2064 - val_accuracy: 0.8487 - val_loss: 0.4109
Epoch 5/5
782/782 ━━━━━━━━━━━━━━━━━━━━ 511s 654ms/step - accuracy: 0.9594 - loss: 0.1218 - val_accuracy: 0.7719 - val_loss: 0.5997


In [14]:
# Evaluar modelo
scores = model.evaluate(x_test, y_test,
                            batch_size = 32,
                            verbose = 2)




782/782 - 112s - 143ms/step - accuracy: 0.7719 - loss: 0.5997


In [21]:
print(f"Accuracy: {scores[1]:.2f}")
print(f"Loss: {scores[0]:.2f}")


Accuracy: 0.86
Loss: 0.60


In [17]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 503, 128)       │     6,400,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 128)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,595,141 (74.75 MB)

 Trainable params: 6,531,713 (24.92 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,063,428 (49.83 MB)

In [30]:
# Guardar modelo
model.save("modelo.keras")

print("Modelo guardado en disco")

Modelo guardado en disco
